notes:
https://www.datacamp.com/community/tutorials/categorical-data

binary encoding for high cardinality ordinal data
high cardinality bad for one hot encoding for decision trees
It’s worth trying HashingEncoder for nominal and ordinal data if you have high cardinality features.

For nominal columns try OneHot, Hashing, LeaveOneOut, and Target encoding. 
Avoid OneHot for high cardinality columns and decision tree-based algorithms.
For ordinal columns try Ordinal (Integer), Binary, OneHot, LeaveOneOut, and Target. Helmert, Sum, BackwardDifference and Polynomial are less likely to be helpful, but if you have time or theoretic reason you might want to try them.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from category_encoders import TargetEncoder


# prepare some features

In [88]:
cars1 = pd.read_csv("/Users/tingjue/170 project/cars2.0.csv",index_col=0)

In [89]:
cars2 = pd.read_csv("/Users/tingjue/170 project/cars3_0.csv",index_col=1)

In [99]:
c = cars2[['vin', 'lstm_feature', 'bilstm_feature','gru_feature','cnn_feature','fast_feature']]

In [100]:
cars =pd.merge(cars1,c,on='vin')

In [101]:
cars = cars.drop(['model_x'],axis = 1)

In [102]:
cars['location']=cars['region']+' '+ cars['state']

In [103]:
cars = cars.drop(['region'], axis = 1)\
.drop(['state'], axis = 1)


In [104]:
cars.to_csv("cars4.csv")

# categorical encoding

In [164]:
df = pd.read_csv("/Users/tingjue/170 project/cars4.csv",index_col=0)

In [133]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59917 entries, 0 to 59916
Data columns (total 23 columns):
id                59917 non-null int64
vin               59917 non-null object
price             59917 non-null int64
year              59917 non-null float64
manufacturer      59917 non-null object
model             59917 non-null object
avg_rating        59917 non-null float64
count             59917 non-null float64
condition         59917 non-null object
cylinders         59917 non-null object
fuel              59917 non-null object
odometer          59917 non-null float64
title_status      59917 non-null object
transmission      59917 non-null object
drive             59917 non-null object
paint_color       59917 non-null object
description       59917 non-null object
lstm_feature      59917 non-null float64
bilstm_feature    59917 non-null float64
gru_feature       59917 non-null float64
cnn_feature       59917 non-null float64
fast_feature      59917 non-null float64
loca

In [165]:
df = df.drop(['vin'],axis=1)\
.drop(['description'],axis=1)

In [163]:
test = df[['condition']]
test = test.sample(10)
encode_condition = {'new': 6, 'like new': 5, 'excellent': 4, 'good': 3,
                                  'fair': 2, 'salvage': 1}

test['encode_condition']=df['condition'].map(encode_condition)
test


,condition,encode_condition
39177,like new,5
1059,good,3
59684,good,3
36208,excellent,4
46643,excellent,4
10726,good,3
39533,good,3
51925,good,3
22868,excellent,4
37038,good,3


In [115]:
##condition
#new, like new, excellent, good, fair, salvage
#ordinal encoding because they are ordinal categorical data
encode_condition = {'condition': {'new': 6, 'like new': 5, 'excellent': 4, 'good': 3,
                                  'fair': 2, 'salvage': 1}}

df.replace(encode_condition, inplace=True)


In [116]:
##cylinders 
#6 cylinders, 4 cylinders,8 cylinders,5 cylinders,10 cylinders,other,3 cylinders,12 cylinders
#ordinal encoding - may change to binary/hash if low corr
encode_cylinders = {'cylinders':{'other': 1,'3 cylinders': 2,'4 cylinders': 3,
                               '5 cylinders': 4,'6 cylinders': 5,'8 cylinders': 6,
                               '10 cylinders': 7,'12 cylinders': 8}}
df.replace(encode_cylinders, inplace=True)

In [117]:
pd.set_option('display.max_columns', None)

In [166]:
##fuel
#'diesel', 'electric', 'gas', 'hybrid', 'other'
# one hot encoding - nominal and low cardinality
encode_fuel = pd.get_dummies(df.fuel)
df = pd.concat([df, encode_fuel], axis=1)
df = df.drop(['fuel'],axis=1)

In [167]:
##title_status
# 'clean', 'salvage','rebuilt','parts only','lien', 'missing'
# frequncy encoding: assign the weight because 95% are clean
encode_title = df.groupby('title_status').size()/len(df)
df.loc[:,'title_status_encode']=df['title_status'].map(encode_title)
df=df.drop(['title_status'],axis=1)

In [168]:
df

,id,price,year,manufacturer,model,avg_rating,count,condition,cylinders,odometer,transmission,drive,paint_color,lstm_feature,bilstm_feature,gru_feature,cnn_feature,fast_feature,location,diesel,electric,gas,hybrid,other,title_status_encode
0,0,17899,2012.0,volkswagen,golf,4.600694,36.0,excellent,4 cylinders,63500.0,manual,4wd,black,18198.558594,27333.046875,22720.089844,23409.597656,21392.708984,salt lake city ut,0,0,1,0,0,0.957608
1,4398,9891,2012.0,volkswagen,golf,4.600694,36.0,excellent,4 cylinders,89419.0,automatic,fwd,white,16551.052734,16830.625000,15969.620117,13576.520508,15707.687500,wenatchee wa,1,0,0,0,0,0.957608
2,23723,7995,2012.0,volkswagen,golf,4.600694,36.0,good,5 cylinders,41600.0,automatic,fwd,grey,12009.676758,9721.872070,11588.109375,6487.177734,5450.783203,new haven ct,0,0,1,0,0,0.957608
3,25205,12900,2012.0,volkswagen,golf,4.600694,36.0,excellent,4 cylinders,68073.0,automatic,fwd,blue,19342.486328,11827.651367,14768.078125,16006.382812,15344.616211,ft myers / sw florida fl,1,0,0,0,0,0.957608
4,33832,21270,2012.0,volkswagen,golf,4.600694,36.0,excellent,4 cylinders,53169.0,automatic,4wd,blue,20155.263672,12482.712891,18049.710938,15435.342773,14372.857422,chicago il,0,0,1,0,0,0.957608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59912,82308,9995,2006.0,volkswagen,touareg,4.305556,72.0,good,10 cylinders,103964.0,automatic,4wd,grey,8014.783691,6944.399414,11832.634766,6348.051270,8534.781250,san antonio tx,1,0,0,0,0,0.957608
59913,82426,2200,2003.0,hyundai,accent,4.259615,52.0,good,4 cylinders,148000.0,manual,fwd,white,4135.699707,4890.686035,5598.274414,2871.909424,3641.207520,san marcos tx,0,0,1,0,0,0.957608
59914,82576,12900,2009.0,toyota,4runner,4.357143,7.0,excellent,6 cylinders,115950.0,automatic,rwd,blue,14413.969727,12452.664062,10286.337891,12750.390625,11489.996094,tyler / east tx tx,0,0,1,0,0,0.957608
59915,82653,2000,2001.0,volvo,s40,3.842391,69.0,excellent,4 cylinders,75000.0,automatic,fwd,white,7259.627441,5160.313477,4470.877930,10385.222656,5719.744629,tyler / east tx tx,0,0,0,0,1,0.957608


In [169]:
##transmission
# one hot encoding - nominal and low cardinality
encode_transmission = pd.get_dummies(df.transmission)
df = pd.concat([df, encode_transmission], axis=1)
df = df.drop(['transmission'],axis=1)

In [171]:
column_names = df.columns.values
column_names[-1] = 'other transmission'
column_names[-5] = 'other fuel'
df.columns = column_names

In [172]:
##drive
# one hot encoding - nominal and low cardinality
encode_drive = pd.get_dummies(df.drive)
df = pd.concat([df, encode_drive], axis=1)
df = df.drop(['drive'],axis=1)


In [176]:
df.drop(['title_status_encode'],axis=1)

,id,price,year,manufacturer,model,avg_rating,count,condition,cylinders,odometer,transmission,paint_color,lstm_feature,bilstm_feature,gru_feature,cnn_feature,fast_feature,location,diesel,electric,gas,hybrid,other fuel,automatic,manual,other transmission,4wd,fwd,rwd
0,0,17899,2012.0,volkswagen,golf,4.600694,36.0,excellent,4 cylinders,63500.0,manual,black,18198.558594,27333.046875,22720.089844,23409.597656,21392.708984,salt lake city ut,0,0,1,0,0,0,1,0,1,0,0
1,4398,9891,2012.0,volkswagen,golf,4.600694,36.0,excellent,4 cylinders,89419.0,automatic,white,16551.052734,16830.625000,15969.620117,13576.520508,15707.687500,wenatchee wa,1,0,0,0,0,1,0,0,0,1,0
2,23723,7995,2012.0,volkswagen,golf,4.600694,36.0,good,5 cylinders,41600.0,automatic,grey,12009.676758,9721.872070,11588.109375,6487.177734,5450.783203,new haven ct,0,0,1,0,0,1,0,0,0,1,0
3,25205,12900,2012.0,volkswagen,golf,4.600694,36.0,excellent,4 cylinders,68073.0,automatic,blue,19342.486328,11827.651367,14768.078125,16006.382812,15344.616211,ft myers / sw florida fl,1,0,0,0,0,1,0,0,0,1,0
4,33832,21270,2012.0,volkswagen,golf,4.600694,36.0,excellent,4 cylinders,53169.0,automatic,blue,20155.263672,12482.712891,18049.710938,15435.342773,14372.857422,chicago il,0,0,1,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59912,82308,9995,2006.0,volkswagen,touareg,4.305556,72.0,good,10 cylinders,103964.0,automatic,grey,8014.783691,6944.399414,11832.634766,6348.051270,8534.781250,san antonio tx,1,0,0,0,0,1,0,0,1,0,0
59913,82426,2200,2003.0,hyundai,accent,4.259615,52.0,good,4 cylinders,148000.0,manual,white,4135.699707,4890.686035,5598.274414,2871.909424,3641.207520,san marcos tx,0,0,1,0,0,0,1,0,0,1,0
59914,82576,12900,2009.0,toyota,4runner,4.357143,7.0,excellent,6 cylinders,115950.0,automatic,blue,14413.969727,12452.664062,10286.337891,12750.390625,11489.996094,tyler / east tx tx,0,0,1,0,0,1,0,0,0,0,1
59915,82653,2000,2001.0,volvo,s40,3.842391,69.0,excellent,4 cylinders,75000.0,automatic,white,7259.627441,5160.313477,4470.877930,10385.222656,5719.744629,tyler / east tx tx,0,0,0,0,1,1,0,0,0,1,0


In [123]:
##paint_color
# frequency encoding - a little bit high cardinality (evenly distributed)
# maybe try target encoding later
encode_color = df.groupby('paint_color').size()/len(df)
df.loc[:,'color_encode']=df['paint_color'].map(encode_color)
df=df.drop(['paint_color'],axis=1)

In [124]:
##location
# frequency encoding
# try later hash encoding - very high cardinality and loss of some info
encode_loc = df.groupby('location').size()/len(df)
df.loc[:,'location_encode']=df['location'].map(encode_loc)
df=df.drop(['location'],axis=1)


In [125]:
##manufacturer 
#target
##try later one-hot encoding 
means =df.groupby('manufacturer').price.mean()
df['manufacturer_encode'] = df['manufacturer'].map(means)
df=df.drop(['manufacturer'],axis=1)

In [126]:
##model 
# frequency
# hash/target try later hash
encode_model = df.groupby('model').size()/len(df)
df.loc[:,'model_encode']=df['model'].map(encode_model)
df=df.drop(['model'],axis=1)

In [128]:
df.to_csv("cars ml.csv")